## 4.이항 분류 모델의 성능평가
- sklearn.metrics.accuracy_score(y_true, y_pred) = (TP/TN) / (TP+TN+FP+FN)
- sklearn.metrics.precision_score(y_true, y_pred) = TP / (TP + FP)
- sklearn.metrics.recall_score(y_true, y_pred) = TP / (TP + FN)
- sklearn.metrics.f1_score(y_true, y_pred = 2 * (Precison * Recall) / (Precision + Recall)
- 참고 : https://gaussian37.github.io/ml-concept-ml-evaluation/

In [56]:
# [0] 성능평가 관련 함수
from sklearn.metrics import confusion_matrix,precision_score,recall_score,accuracy_score,f1_score
import numpy as np
import pandas as pd

y_true = np.array([0,1,0,0]) # 0:3개, 1:1개, 불균형한 데이터
y_pred = np.array([0,1,1,0])
cm = pd.DataFrame(confusion_matrix(y_true,y_pred))
print(cm)
#전체에서 맞춘것의 비율 : 3/4
print('accuracy : %.2f' %accuracy_score(y_true,y_pred))
#positive(1)로 예측한 것 중에 맞춘것 : 1/2
print('precision : %.2f' %precision_score(y_true,y_pred))
#positive(1) 가 실제인 것 중에 맞춘것 : 1/1
print('recall : %.2f' %recall_score(y_true,y_pred))
# f1 : precision,recall의 조화 평균
print('f1 : %.2f' %f1_score(y_true,y_pred))

   0  1
0  2  1
1  0  1
accuracy : 0.75
precision : 0.50
recall : 1.00
f1 : 0.67


In [57]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

In [58]:
!git clone https://github.com/Soyoung-Yoon/bigdata

from sklearn.model_selection import train_test_split
dftot = pd.read_csv('bigdata/1st_Train.csv')

x_train,x_test = train_test_split(dftot,test_size=0.4,
                                                 stratify=dftot['Reached.on.Time_Y.N'],random_state=0)
y_train = x_train[['ID','Reached.on.Time_Y.N']]
x_train = x_train.drop(columns='Reached.on.Time_Y.N')
y_test = x_test[['ID','Reached.on.Time_Y.N']]
x_test = x_test.drop(columns='Reached.on.Time_Y.N')

x_train.to_csv('x_train.csv',index=False)
y_train.to_csv('y_train.csv',index=False)
x_test.to_csv('x_test.csv',index=False)
y_test.to_csv('y_test.csv',index=False)

X = pd.read_csv('x_train.csv')
Y = pd.read_csv('y_train.csv')
X_submission = pd.read_csv('x_test.csv')


dfX = pd.concat([X,X_submission],axis=0,ignore_index=True)

df_LE = dfX.copy()
df_LE['Gender'] = dfX['Gender'].replace(['F','M'],['0','1']).astype('int64')
df_LE['Warehouse_block'] = dfX['Warehouse_block'].replace(['A','B','C','D','F'],['0','1','2','3','4']).astype('int64')
df_LE['Mode_of_Shipment'] = dfX['Mode_of_Shipment'].replace(['Ship','Flight','Road'],['0','1','2']).astype('int64')
df_LE['Product_importance'] = dfX['Product_importance'].replace(['low','medium','high'],['0','1','2']).astype('int64')

# train socre,test score,roc_auc_score 를 반환하는 함수 작성
def get_scores(model,xtrain,xtest,ytrain,ytest):
  A = model.score(xtrain,ytrain)
  B = model.score(xtest,ytest)
  ypred = model.predict_proba(xtest)[:,1]
  C = roc_auc_score(ytest,ypred)
  return '{:.4f} {:.4f} {:.4f}'.format(A,B,C)


# X를 train용도,submission용도로 나누고 ,Y를 1차원으로 바꿈(MinMaxScaler 적용)
def get_data(dfX,Y):
  X = dfX.drop(columns=['ID'])
  X_use = X.iloc[:6599,:]        # 훈련 데이터 (앞부분)
  X_submission = X.iloc[6599:,:] # 제출용 데이터 (뒷부분)
  Y1 = Y['Reached.on.Time_Y.N']
  scaler = MinMaxScaler()
  X1_use = scaler.fit_transform(X_use)
  X1_submission = scaler.fit_transform(X_submission)
  print(X1_use.shape,X1_submission.shape,Y1.shape)
  return X1_use,X1_submission,Y1

# df_LE 사용
#X,Y 분리하기
X1_use,X1_submission,Y1 = get_data(df_LE,Y)
#train,test 7:3분할, stratify 적용, random_state=0 적용
xtrain1,xtest1,ytrain1,ytest1 = train_test_split(X1_use,Y1,
                                                  test_size=0.3,
                                                  stratify = Y1,
                                                  random_state=0)

fatal: destination path 'bigdata' already exists and is not an empty directory.
(6599, 10) (4400, 10) (6599,)


In [59]:
# [1] 데이터를 사용한 성능 출력 함수 만들기
#     accuracy, precision, recall, f1
def get_other_scores(model,x_test,y_test):
  y_pred = model.predict(x_test)
  accuracy = accuracy_score(y_test,y_pred)
  precision = precision_score(y_test,y_pred)
  recall = recall_score(y_test,y_pred)
  f1 = f1_score(y_test,y_pred)
  print(f'accuracy :{accuracy:7.4f}')
  print(f'precision : {precision:7.4f}')
  print(f'recall : {recall:7.4f}')
  print(f'f1 : {f1:7.4}')

In [60]:
# [2] DecisionTreeClassifier - max_depth를 5로 했을 경우
model_1 = DecisionTreeClassifier(max_depth=5,random_state=0).fit(xtrain1,ytrain1)
print('model',get_scores(model_1,xtrain1,xtest1,ytrain1,ytest1))
get_other_scores(model_1,xtest1,ytest1)

model 0.6893 0.6874 0.7487
accuracy : 0.6874
precision :  0.9490
recall :  0.5034
f1 :  0.6578


In [61]:
# [3] DecisionTreeClassifier - max_depth를 4로 했을 경우
model_2 = DecisionTreeClassifier(max_depth=4,random_state=0).fit(xtrain1,ytrain1)
print('model',get_scores(model_2,xtrain1,xtest1,ytrain1,ytest1))
get_other_scores(model_2,xtest1,ytest1)

model 0.6820 0.6783 0.7437
accuracy : 0.6783
precision :  0.9549
recall :  0.4839
f1 :  0.6423
